# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint
import numpy as np
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,twin falls,42.3505,-114.6445,49.89,76,0,10.36,US,1706662114
1,1,takoradi,4.8845,-1.7554,79.45,83,14,6.78,GH,1706662115
2,2,papatowai,-46.5619,169.4708,65.62,58,100,7.31,NZ,1706662115
3,3,lorengau,-2.0226,147.2712,83.39,81,100,10.85,PG,1706662115
4,4,college,64.8569,-147.8028,-14.85,64,75,0.00,US,1706662115


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM", 
                    frame_width = 700, frame_height = 500, 
                    size = "Humidity", color = "City", cmap = 'ocean')

# Display the map
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] > 70) & 
                                   (city_data_df["Max Temp"] < 90) & 
                                   (city_data_df["Wind Speed"] < 20) & 
                                   (city_data_df["Cloudiness"] == 0), :]
# Drop any rows with null values
clean_df = ideal_weather_df.dropna(how='any')
clean_df.reset_index(inplace=True)
del clean_df['index']

# Display sample data
clean_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,30,barra de santo antonio,-9.4047,-35.5072,76.28,79,0,7.31,BR,1706662120
1,43,juan de ayolas,-27.4000,-56.9000,77.81,58,0,8.14,PY,1706662122
2,54,margaret river,-33.9500,115.0667,76.82,47,0,5.70,AU,1706662124
3,222,bayeux,-7.1250,-34.9322,79.43,89,0,4.61,BR,1706662153
4,228,piacabucu,-10.4056,-36.4344,78.06,78,0,7.56,BR,1706662154
5,230,port augusta,-32.5000,137.7667,73.31,34,0,18.90,AU,1706662154
6,271,mount gambier,-37.8333,140.7667,70.30,48,0,9.04,AU,1706661966
7,281,sao luis,-2.5297,-44.3028,82.60,89,0,10.36,BR,1706662164
8,287,inhambane,-23.8650,35.3833,74.19,87,0,9.48,MZ,1706662165
9,309,ebaye,8.7748,167.7392,82.42,74,0,18.41,MH,1706662169


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,twin falls,US,42.3505,-114.6445,76,
1,takoradi,GH,4.8845,-1.7554,83,
2,papatowai,NZ,-46.5619,169.4708,58,
3,lorengau,PG,-2.0226,147.2712,81,
4,college,US,64.8569,-147.8028,64,
...,...,...,...,...,...,...
594,prince george,CA,53.9166,-122.7530,61,
595,betare oya,CM,5.6000,14.0833,27,
596,laughlin,US,35.1678,-114.5730,30,
597,olyokminsk,RU,60.3736,120.4264,100,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = {"apiKey": geoapify_key,
         "categories":categories}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filters"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
twin falls - nearest hotel: The Fillmore Inn
takoradi - nearest hotel: Hillcrest
papatowai - nearest hotel: Catlins Woodstock Lodge & Camp
lorengau - nearest hotel: Seeadler Bay Hotel
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
nova vicosa - nearest hotel: Pousada Pontal da Barra
egilsstadir - nearest hotel: Valaskjálf
edinburgh of the seven seas - nearest hotel: No hotel found
bethel - nearest hotel: Hampton Inn Danbury
gokarna - nearest hotel: Rudraksh Retreat
waitangi - nearest hotel: Hotel Chathams
mahina - nearest hotel: Motu Martin
tvoroyri - nearest hotel: Hotel Tvøroyri
adamstown - nearest hotel: Chez Bianca & Benoit
iqaluit - nearest hotel: Frobisher Inn
port-aux-francais - nearest hotel: Keravel
hammerfest - nearest hotel: Thon Hotel Hammerfest
bamboo flat - nearest hotel: Hotel Megapode Nest
kone - nearest hotel: Pacifik Appartels
longyearbyen - nearest hotel: The Vault
bengkulu - nearest hotel: Dena Hotel
dolores hidalgo cuna de la inde

,City,Country,Lat,Lng,Humidity,Hotel Name
0,twin falls,US,42.3505,-114.6445,76,The Fillmore Inn
1,takoradi,GH,4.8845,-1.7554,83,Hillcrest
2,papatowai,NZ,-46.5619,169.4708,58,Catlins Woodstock Lodge & Camp
3,lorengau,PG,-2.0226,147.2712,81,Seeadler Bay Hotel
4,college,US,64.8569,-147.8028,64,BEST WESTERN PLUS Pioneer Park Inn
...,...,...,...,...,...,...
594,prince george,CA,53.9166,-122.7530,61,Travelodge by Wyndham Prince George
595,betare oya,CM,5.6000,14.0833,27,Motel le Kadey
596,laughlin,US,35.1678,-114.5730,30,Don Laughlin's Riverside Resort
597,olyokminsk,RU,60.3736,120.4264,100,Колос


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot_hotel = hotel_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM", 
                                frame_width = 700, frame_height = 500, 
                                size = "Humidity", color = "City", 
                                hover_cols = ["Country", "Hotel Name"], cmap = 'ocean')

# Display the map
map_plot_hotel

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)